<a href="https://colab.research.google.com/github/naveen333nav/LabWorkRepo/blob/master/Umpire_sign_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive') 

In [0]:
!cp -r "/content/gdrive/My Drive/Inception_test_train_split" "/content/"

In [0]:
%tensorflow_version 1.x
import tensorflow as tf

In [0]:
!wget "https://github.com/aaravindravi/A-Dataset-and-Preliminary-Results-for-Umpire-Pose-Detection-Using-SVM-Classification-of-Deep-Feature/archive/master.zip"

In [0]:
"""
Feature extraction for model 1 using Inception
Paper: Ravi, Aravind, Harshwin Venugopal, Sruthy Paul, and Hamid R. Tizhoosh. 
"A Dataset and Preliminary Results for Umpire Pose Detection Using SVM Classification of Deep Features." 
arXiv preprint arXiv:1809.06217 (2018).

"""

import os
import numpy as np
import glob
from keras import applications
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image
from keras import applications
from keras.models import Model
import time


#Download the base model as Inception V3
base_model = applications.inception_v3.InceptionV3(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output)


start_time = time.time()

curr_cwd = "/content/Inception_test_train_split/model1_train" #path to the Dataset folder


#SNOW Dataset 
for cls in range (1,3):
    #Folders containing the dataset
    classes = ["non_umpire","umpire"]
    

    img_count=0
    feats=[]
    #Extract the output of the avg_pool layer of inception
    
    for image_path in glob.glob(curr_cwd+"/"+classes[cls-1]+"*.jpg"):
        
        img_count=img_count+1
        print(img_count)
        # load image and set image size to 299 x 299
        img = image.load_img(image_path, target_size=(299, 299))
        # convert image to numpy array
        x = image.img_to_array(img)
        # convert the image into array of shape (3, 299, 299) 
        # need to expand it to (1, 3, 299, 299)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        # extracting the features from the given layer
        features = model.predict(x)
        
        feats.append(features)
        features_arr = np.char.mod('%f', features)
    
    feature_list = np.squeeze(np.asarray(feats))
    #storing the labels based on the class number
    labels = np.ones(len(feature_list))*cls
    feature_list = np.column_stack((feature_list,labels))
    #Save the features as a numpy array for further processing
    np.save("class_"+str(cls)+"data_model1.npy",feature_list)
     
     
print("--- %s seconds ---" % (time.time() - start_time))

In [0]:
!ls

In [0]:

"""
Feature extraction for model2 using Inception
Paper: Ravi, Aravind, Harshwin Venugopal, Sruthy Paul, and Hamid R. Tizhoosh. 
"A Dataset and Preliminary Results for Umpire Pose Detection Using SVM Classification of Deep Features." 
arXiv preprint arXiv:1809.06217 (2018).

"""

import os
import numpy as np
import glob
from keras import applications
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image
from keras import applications
from keras.models import Model
import time

#Download the base model as Inception V3
base_model = applications.inception_v3.InceptionV3(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output)

#Get current working directory
curr_cwd = "/content/Inception_test_train_split/model2_train" #path to the Dataset

start_time = time.time()


#SNOW Dataset 
for cls in range (1,6):
    #Folders containing the dataset
    classes = ["no_ball","out","sixes","wide","no_action"]

    img_count=0
    feats=[]
    #Extract the output of the avg_pool layer of inception
    
    for image_path in glob.glob(curr_cwd+"/"+classes[cls-1]+"*.jpg"):
        img_count=img_count+1
        print(img_count)
        # load image and set image size to 299 x 299
        img = image.load_img(image_path, target_size=(299, 299))
        # convert image to numpy array
        x = image.img_to_array(img)
        # convert the image into array of shape (3, 299, 299) 
        # need to expand it to (1, 3, 299, 299)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        # extracting the features from the given layer
        features = model.predict(x)
        
        feats.append(features)
        features_arr = np.char.mod('%f', features)
    
    feature_list = np.squeeze(np.asarray(feats))
    #storing the labels based on the class number
    labels = np.ones(len(feature_list))*cls
    feature_list = np.column_stack((feature_list,labels))
    #Save the features as a numpy array for further processing
    np.save("class_"+str(cls)+"data.npy",feature_list)
     
print("--- %s seconds ---" % (time.time() - start_time))
     


In [25]:

"""
SVM classifier for model 1 using inception
Paper: Ravi, Aravind, Harshwin Venugopal, Sruthy Paul, and Hamid R. Tizhoosh. 
"A Dataset and Preliminary Results for Umpire Pose Detection Using SVM Classification of Deep Features." 
arXiv preprint arXiv:1809.06217 (2018).

"""
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import LeaveOneOut
import pickle
import time

#Storing the extracted features
X=[]
X1 = np.load('class_1data_model1.npy')
X2 = np.load('class_2data_model1.npy')
X_data = np.append(X1,X2,axis=0)

#Labels
Y_data = X_data[:,(X_data.shape[1]-1)]

#Training Data
X_data = X_data[:,0:(X_data.shape[1]-1)]

x_tr,x_ts,y_tr,y_ts = train_test_split(X_data, Y_data, test_size=0.2,random_state=10)

#Classifier using Linear SVM

clf = LinearSVC(C=10)
start_time = time.time()
clf = clf.fit(x_tr,y_tr)
predictions_tr = (clf.predict(x_ts))

#10-fold Cross-Validation    
scores = cross_val_score(clf, x_tr, y_tr, cv=10)
test_acc = accuracy_score(y_ts,predictions_tr)

print("Training Accuracy: %0.4f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print("Test Accuracy: %0.4f" % test_acc)
print("--- %s seconds ---" % (time.time() - start_time))


#Leave One Out Validation
loo_train_acc=[]
loo = LeaveOneOut()
for train_index, test_index in loo.split(x_tr):
   #print("TRAIN:", train_index, "TEST:", test_index)
   X_train, X_test = x_tr[train_index], x_tr[test_index]
   y_train, y_test = y_tr[train_index], y_tr[test_index]
   clf = clf.fit(X_train,y_train)
   predictions = (clf.predict(X_test))
   loo_train_acc.append(accuracy_score(y_test,predictions))

loo_train_accuracy = np.asarray(loo_train_acc)
print("LOO Accuracy: %0.4f" % loo_train_accuracy.mean())

#Save the model
pickle.dump(clf, open('cricket_inceptionv3_equal_linsvm_model1.sav', 'wb'))


Training Accuracy: 0.9370 (+/- 0.11)
Test Accuracy: 0.9500
--- 1.751107931137085 seconds ---
LOO Accuracy: 0.9367


In [26]:

"""
Svm classifier for model 2 using inception
Paper: Ravi, Aravind, Harshwin Venugopal, Sruthy Paul, and Hamid R. Tizhoosh. 
"A Dataset and Preliminary Results for Umpire Pose Detection Using SVM Classification of Deep Features." 
arXiv preprint arXiv:1809.06217 (2018).

"""
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.model_selection import LeaveOneOut
import pickle
import time


#Storing the extracted features
X=[]
X1 = np.load('class_1data.npy')
X2 = np.load('class_2data.npy')
X3 = np.load('class_3data.npy')
X4 = np.load('class_4data.npy')
X5 = np.load('class_5data.npy')

X_data = np.append(X1,X2,axis=0)
X_data = np.append(X_data,X3,axis=0)
X_data = np.append(X_data,X4,axis=0)
X_data = np.append(X_data,X5,axis=0)

#Labels
Y_data = X_data[:,(X_data.shape[1]-1)]

#Training Data
X_data = X_data[:,0:(X_data.shape[1]-1)]

#Split the data into 80% training and 20% test data
x_tr,x_ts,y_tr,y_ts = train_test_split(X_data, Y_data, test_size=0.2,random_state=26)

#Classifier using Linear SVM

clf = LinearSVC(C=10)
start_time = time.time()
clf = clf.fit(x_tr,y_tr)
predictions_tr = (clf.predict(x_ts))

#10-fold Cross-Validation    
scores = cross_val_score(clf, x_tr, y_tr, cv=10)
test_acc = accuracy_score(y_ts,predictions_tr)

print("Training Accuracy: %0.4f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
print("Test Accuracy: %0.4f" % test_acc)
print("--- %s seconds ---" % (time.time() - start_time))


##Leave One Out Validation
loo_train_acc=[]
loo = LeaveOneOut()
for train_index, test_index in loo.split(x_tr):
   print("TRAIN:", train_index, "TEST:", test_index)
   X_train, X_test = x_tr[train_index], x_tr[test_index]
   y_train, y_test = y_tr[train_index], y_tr[test_index]
   clf = clf.fit(X_train,y_train)
   predictions = (clf.predict(X_test))
   loo_train_acc.append(accuracy_score(y_test,predictions))

loo_train_accuracy = np.asarray(loo_train_acc)
print("LOO Accuracy: %0.4f" % loo_train_accuracy.mean())

#Save the model
pickle.dump(clf, open('cricket_inceptionv3_equal_linsvm.sav', 'wb'))



/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarn

Training Accuracy: 0.8150 (+/- 0.16)
Test Accuracy: 0.7937
--- 26.955595016479492 seconds ---
TRAIN: [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 22

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


TRAIN: [  0   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 245

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


...

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


TRAIN: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244

/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [28]:
!wget "http://dl152.y2mate.com/?file=M3R4SUNiN3JsOHJ6WWQ2a3NQS1Y5ZGlxVlZIOCtyZ1hqTkFzd2xzQkZLaEhxSUk5a3JiM01JSjRJYlVjMmNHWEhNNWQ1emZXZHB1dk5ReUFwWkVwVEdESzBzQTRxM1NDdUtZcFhzRjlYeEh3ajYyQW1UeG1nQXYwYXRIQ0cvTWJKbFJwcWtObzFqTFkrZnpFc0FmeHZYcVN2bXZRWHpkWXQySWdIdjd2b3NrWXhTZkdmNk9pMVprSnFCbVA0NDVCaDdYUGtoajUxcVlydXRrd0RHSjVZOFFZaTd5bW1xTHF2QlVVZ1kwWmxFNnJwS25yVFkwaEdiT2JSalZsUFNrSDV2MzhZd3dSM1RZS3gzbXo1cVkzNTI0T1RxUnc2MnVuOFAzbmRRQ2RhTWZrY0ptTkxlQ3hyOEx4OXNaaXNFelJyKy9HaUpOUG1EZWtXc0g4VGRRY2psb3VzYXFSN01JeC9VK2ZpVHdKME9WUTcyajNTQjlySG9jRkxtZGJZYzhiRGlzV3YrVFN0UFJvcmNZSlBXMjNzK3RoTXFrYzljZWNoT2s2bEN1SDlacHBFVnVWNEhMSkQ2NnNDREF1OTJWeUFtQ0JFZ2dYL2c1ek9rZW5oNDlueEhDc0dvQWdJR0ljMVd2c0VsTGFYK0E1"

The name is too long, 672 chars total.
Trying to shorten...
New name is index.html?file=M3R4SUNiN3JsOHJ6WWQ2a3NQS1Y5ZGlxVlZIOCtyZ1hqTkFzd2xzQkZLaEhxSUk5a3JiM01JSjRJYlVjMmNHWEhNNWQ1emZXZHB1dk5ReUFwWkVwVEdESzBzQTRxM1NDdUtZcFhzRjlYeEh3ajYyQW1UeG1nQXYwYXRIQ0cvTWJKbFJwcWtObzFqTFkrZnpFc0FmeHZYcVN2bXZRWHpkWXQySWdI.
--2020-02-18 16:53:30--  http://dl152.y2mate.com/?file=M3R4SUNiN3JsOHJ6WWQ2a3NQS1Y5ZGlxVlZIOCtyZ1hqTkFzd2xzQkZLaEhxSUk5a3JiM01JSjRJYlVjMmNHWEhNNWQ1emZXZHB1dk5ReUFwWkVwVEdESzBzQTRxM1NDdUtZcFhzRjlYeEh3ajYyQW1UeG1nQXYwYXRIQ0cvTWJKbFJwcWtObzFqTFkrZnpFc0FmeHZYcVN2bXZRWHpkWXQySWdIdjd2b3NrWXhTZkdmNk9pMVprSnFCbVA0NDVCaDdYUGtoajUxcVlydXRrd0RHSjVZOFFZaTd5bW1xTHF2QlVVZ1kwWmxFNnJwS25yVFkwaEdiT2JSalZsUFNrSDV2MzhZd3dSM1RZS3gzbXo1cVkzNTI0T1RxUnc2MnVuOFAzbmRRQ2RhTWZrY0ptTkxlQ3hyOEx4OXNaaXNFelJyKy9HaUpOUG1EZWtXc0g4VGRRY2psb3VzYXFSN01JeC9VK2ZpVHdKME9WUTcyajNTQjlySG9jRkxtZGJZYzhiRGlzV3YrVFN0UFJvcmNZSlBXMjNzK3RoTXFrYzljZWNoT2s2bEN1SDlacHBFVnVWNEhMSkQ2NnNDREF1OTJWeUFtQ0JFZ2dYL2c1ek9rZW5oNDlueEhDc0dvQWd

In [31]:

"""
Video summarization using Inception
Paper: Ravi, Aravind, Harshwin Venugopal, Sruthy Paul, and Hamid R. Tizhoosh. 
"A Dataset and Preliminary Results for Umpire Pose Detection Using SVM Classification of Deep Features." 
arXiv preprint arXiv:1809.06217 (2018).
"""
import os
import cv2
from keras import applications
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image
import numpy as np  
import pickle
from keras.models import Model
import time

start_time = time.time()
base_model = applications.inception_v3.InceptionV3(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)
model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output)

#Project Model- Loading the saved Inception models
loaded_model1 = pickle.load(open('cricket_inceptionv3_equal_linsvm_model1.sav', 'rb'))
loaded_model2 = pickle.load(open('cricket_inceptionv3_equal_linsvm.sav', 'rb'))

#Path for the video to be summarized
vidcap = cv2.VideoCapture('testVideoAll1.mp4')

count = 0
bufferCount = 0

globalWideVideo =[]
globalOutVideo =[]
globalSixVideo =[]
globalNoBallVideo =[]
globalNoActionVideo =[]
buffer=[]

th=5
buff_th=250
globalWideCounter = 0
globalOutCounter = 0
globalSixCounter = 0
globalNoBallCounter = 0
globalNoActionCounter = 0


while (True):
    success,img = vidcap.read()
    
    if success:
        bufferCount = bufferCount + 1
        buffer.append(img)
        height, width, layers = img.shape
        size = (width,height)
        count=count+1
        print ("success")
        img1 = cv2.resize(img,(299,299))
        x = image.img_to_array(img1)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        #Feature Extraction Step
        features = model.predict(x) #Inception V3 Model
        #Classification Step
        predicted_values = loaded_model1.predict(features.reshape(1,-1)) 
        if predicted_values==2:
            predicted_values_2 = loaded_model2.predict(features.reshape(1,-1))
            choices = {'1':'noball', '2':'out', '3':'six', '4':'wide', '5':'noaction'}
            result = choices.get(np.str(int(predicted_values_2)), 'default')
            if result == 'noball':
                globalNoBallCounter = globalNoBallCounter + 1
                print('noball:')
            if result == 'out':
                globalOutCounter = globalOutCounter + 1
                print('out:')
            if result == 'six':
                globalSixCounter = globalSixCounter + 1
                print('six:')
            if result == 'wide':
                globalWideCounter = globalWideCounter + 1
                print('wide:')
            if result == 'noaction':
                globalNoActionCounter = globalNoActionCounter + 1
                print('noaction:')

            
    else:
        break
    ## Frame accumulation
    if bufferCount == buff_th:
        actionCount = {'noball': globalNoBallCounter, 'out': globalOutCounter, 'six': globalSixCounter, 'wide': globalWideCounter}

        winner = max(actionCount, key=actionCount.get)
        if winner == 'noball' and globalNoBallCounter >th:
            globalNoBallVideo.append(buffer)
        if winner == 'out'and globalOutCounter >th:
            globalOutVideo.append(buffer) 
        if winner == 'six'and globalSixCounter >th:
            globalSixVideo.append(buffer)
        if winner == 'wide' and globalWideCounter >th:
            globalWideVideo.append(buffer)
       
        bufferCount = 0
        globalWideCounter = 0
        globalOutCounter = 0
        globalSixCounter = 0
        globalNoBallCounter = 0
        globalNoActionCounter = 0
        buffer = []
    
actionCount = {'noball': globalNoBallCounter, 'out': globalOutCounter, 'six': globalSixCounter, 'wide': globalWideCounter}
winner = max(actionCount, key=actionCount.get)
if winner == 'noball' and globalNoBallCounter >th:
    globalNoBallVideo.append(buffer)
if winner == 'out'and globalOutCounter >th:
    globalOutVideo.append(buffer) 
if winner == 'six'and globalSixCounter >th:
    globalSixVideo.append(buffer)
if winner == 'wide' and globalWideCounter >th:
    globalWideVideo.append(buffer)

        
cv2.destroyAllWindows()

print ('Summarizing Video...')


if globalNoBallVideo!=[]:
    noBallVideo = cv2.VideoWriter('no_ball.avi',cv2.VideoWriter_fourcc(*'DIVX'), 25, size)
    for i in range(len(globalNoBallVideo)):
        for j in range(len(globalNoBallVideo[i])):
            # writing to a image array
            noBallVideo.write(globalNoBallVideo[i][j])
    noBallVideo.release()


if globalOutVideo!=[]:
    outVideo = cv2.VideoWriter('out.avi',cv2.VideoWriter_fourcc(*'DIVX'), 25, size)
    for i in range(len(globalOutVideo)):
        for j in range(len(globalOutVideo[i])):
            # writing to a image array
            outVideo.write(globalOutVideo[i][j])
    outVideo.release()    
    


if globalWideVideo!=[]:
    wideVideo = cv2.VideoWriter('wide.avi',cv2.VideoWriter_fourcc(*'DIVX'), 25, size)
    for i in range(len(globalWideVideo)):
        for j in range(len(globalWideVideo[i])):
            # writing to a image array
            wideVideo.write(globalWideVideo[i][j])
    wideVideo.release()
    

if globalSixVideo!=[]:
    sixVideo = cv2.VideoWriter('sixes.avi',cv2.VideoWriter_fourcc(*'DIVX'), 25, size)
    for i in range(len(globalSixVideo)):
        for j in range(len(globalSixVideo[i])):
            # writing to a image array
            sixVideo.write(globalSixVideo[i][j])
    sixVideo.release()
    
print("--- %s seconds ---" % (time.time() - start_time))




/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`


Streaming output truncated to the last 5000 lines.
success
success
success
success
success
success
success
wide:
success
wide:
success
noball:
success
noaction:
success
noaction:
success
success
success
success
noaction:
success
success
noaction:
success
noaction:
success
noaction:
success
noaction:
success
success
noaction:
success
noaction:
success
noaction:
success
success
success
success
success
success
success
success
success
wide:
success
wide:
success
success
noaction:
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
noball:
success
noball:
success
noball:
success
noball:
success
noball:
success
success
success
success
success
success
success
noball:
success
six:
success
success
success
success
success
success
wide:
success
success
success
success
success
success
success
success
success
success
success
noaction:
success
noaction:
success
noaction:
success
noaction:
succ